Example of how to run the LADDIE model and to plot output

In [ ]:
import sys
sys.path.append('../../laddie/src/')
sys.path.append('../src2/')

import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo

from laddie import Laddie
from geometry2 import Geometry
from forcing2 import Forcing

import preprocess as pp
import icedyn as id

from tools import im_,jm_

import warnings
warnings.filterwarnings('ignore')
np.seterr(all='ignore')

#%matplotlib widget
#%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

Prepare model

In [ ]:
reg = 'Thwaites'
SMB = 1.0

geom = Geometry(reg)
geom.coarsen(N=1)
geom = geom.create()

forc = Forcing(geom).sose('R2')
#forc = Forcing(geom).tanh(Tdeep=0.5,ztcl=-500)

laddie = Laddie(forc)

In [ ]:
#Parameters

#Bunch of settings and parameters

laddie.Ah = 24 #Horizontal viscosity
laddie.Kh = 24 #Horizontal diffusivity

laddie.Cdtop = 1.1e-3 #Top drag coefficient
laddie.minD = 4. #Minimum layer thickness
laddie.dt = 96 #Time step
laddie.nu = 0.6

laddie.slip = 1.
laddie.utide = .01
laddie.convop = 1
laddie.vcut = 1.

laddie.SMB = 1.0
laddie.calvlim = 100
laddie.correctisf=False
laddie.nui = 1.e6

laddie.nglen = 3
laddie.Aglen = 2e-17

laddie.diagday = .1

laddiedays = 1 # number of days to integrate laddie for each ice time step
laddie.restday = 1
laddie.saveday = 1

laddiespinup = 10 # number of days to spin up laddie


In [ ]:
#Spin up of ocean

ds = laddie.compute(days=laddiespinup)
#ds = laddie.compute(days=1,restartfile=f'{forc.name_forcing}_009')

In [ ]:
#Prepare ice model

#Get observed ice velocities on u- and v-grid
id.interpolate_icevel(laddie)

#Apply calving if floating ice with thickness below calvlim
id.apply_calving(laddie)

#Create ice mask and masks for ice U and V
id.create_imask(laddie)

#Spin up of ice model
id.prepare_SSA(laddie)
id.integrate_ice(laddie,calcdam=False)

In [ ]:
#Some parameters
dtice = .5 # ice time step in years (effectively the update-frequency for melt calculation and save-frequency of ice variables)
Nyears = 40 # Number of years to integrate
Ntice = int(Nyears/dtice) #Number of time steps for ice
Nbetween = 4 #Intermediate calculations (reduce actual computing time step for ice)

#Restart from a previously saved year to get mask and geometry from that
restyear = 10
dsav   = xr.open_dataset(f'../results/{geom.name_geo.values}_{forc.name_forcing}_{restyear:03.0f}.nc')
laddie.mask = dsav.mask
laddie.zb = dsav.zb
oldmask = laddie.mask.copy()
dsav.close()

#Run laddie for 1 day to get melt rates
ds = laddie.compute(days=1,restartfile=f'{forc.name_forcing}_{restyear:03.0f}')

#Initialise some parms
laddie.sfac = 0.*laddie.isf #fraction of ice shelf cells at ice shelf front
laddie.H = geom.thickness #Should get this from ds or dsav

for n in range(Ntice):

    for m in range(Nbetween):
        #Calculate ice velocities
        id.integrate_ice(laddie,calcdam=False,errlim0=16000,errlim1=1600)

        #Compute ice convergence
        convN = -(np.maximum(laddie.Vssa[0,:,:],0)*laddie.H + np.minimum(laddie.Vssa[0,:,:],0)*np.roll(laddie.H*laddie.imask,-1,axis=0))/laddie.dy
        convS = (np.maximum(np.roll(laddie.Vssa[0,:,:],1,axis=0),0)*np.roll(laddie.H*laddie.imask,1,axis=0) + np.minimum(np.roll(laddie.Vssa[0,:,:],1,axis=0),0)*laddie.H)/laddie.dy
        convE = -(np.maximum(laddie.Ussa[0,:,:],0)*laddie.H + np.minimum(laddie.Ussa[0,:,:],0)*np.roll(laddie.H*laddie.imask,-1,axis=1))/laddie.dx
        convW = (np.maximum(np.roll(laddie.Ussa[0,:,:],1,axis=1),0)*np.roll(laddie.H*laddie.imask,1,axis=1) + np.minimum(np.roll(laddie.Ussa[0,:,:],1,axis=1),0)*laddie.H)/laddie.dx
        conv = convN+convS+convE+convW * laddie.imask

        #Compute new ice thickness
        melt = laddie.lastmelt #Lastmelt = last melt rate averaged over a save-period
        dH = conv-melt + laddie.SMB #Thickness change in m/yr
        Hnew = laddie.H+dH*dtice/Nbetween*laddie.tmask #Integrate H, only integrate floating ice (tmask) to keep 

        #Advect ice into partial grid cells
        """Check: likely need to shift U and V at isf points"""
        laddie.sfac += (-np.minimum(0,laddie.isfW*np.roll(laddie.tmask*laddie.Ussa[0,:,:],-1,axis=1))/laddie.dx \
                +  np.maximum(0,laddie.isfE*np.roll(laddie.tmask*laddie.Ussa[0,:,:],1,axis=1))/laddie.dx \
                +  -np.minimum(0,laddie.isfS*np.roll(laddie.tmask*laddie.Vssa[0,:,:],-1,axis=0))/laddie.dy \
                +  np.maximum(0,laddie.isfN*np.roll(laddie.tmask*laddie.Vssa[0,:,:],1,axis=0))/laddie.dy \
                ) * dtice/Nbetween

        #Adjust mask at ice shelf front, where partial cell is full (sfac>1)
        laddie.mask = np.where(laddie.sfac>1,3,laddie.mask)
        Hnew = np.where(laddie.sfac>1,(laddie.isfW*np.roll(Hnew,-1,axis=1)+laddie.isfE*np.roll(Hnew,1,axis=1)+laddie.isfS*np.roll(Hnew,-1,axis=0)+laddie.isfN*np.roll(Hnew,1,axis=0))/laddie.isf,Hnew)
        pp.create_mask(laddie)
        id.create_imask(laddie)
        laddie.sfac = np.where(laddie.imask,0,laddie.sfac) #Restore fraction to 0 of filled cell. May want to keep this 1 for consistency

        #Prevent circular boundary conditions, not sure this works very well, or whether it's necessary at all
        '''Check whether this actually works as it should'''
        Hnew[0,:] = np.where(laddie.tmask[0,:]==1,0,Hnew[0,:])
        Hnew[-1,:] = np.where(laddie.tmask[-1,:]==1,0,Hnew[-1,:])
        Hnew[:,0] = np.where(laddie.tmask[:,0]==1,0,Hnew[:,0])
        Hnew[:,-1] = np.where(laddie.tmask[:,-1]==1,0,Hnew[:,-1])

        #Overwrite ice thickness
        laddie.H = Hnew

        #Calving routine
        id.apply_calving(laddie)

        #Recompute ice draft zb and ice surface zs from new thickness H
        zbnew = xr.where(laddie.tmask==1,-laddie.rhoi/laddie.rho0*laddie.H,laddie.zb)
        zbnew = xr.where(np.isnan(zbnew),laddie.zb,zbnew) #Dont think this is necessary
        laddie.zb = zbnew
        laddie.zs = np.where(laddie.tmask==1,(1-laddie.rhoi/laddie.rho0)*laddie.H,laddie.zs)

    #Integrate Laddie for next ice time step
    ds = laddie.compute(days=laddiedays,restartfile=laddie.restartfile)